# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

**Initialize Workspace**

In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

wsaug
RGML
centralindia
d18bf486-2340-49bd-9908-fbd241f929a1


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

The dataset for Fraud detection in card payment is downloaded from Kaggle. It's a single CSV file with nearly 1,50,000 records. A dataset named KaggleDSFraudDS is created in workspace based on that. This dataset is used to predict if the given transaction is legit or fraud. The model is trained using LogisticRegression algorithm.

In [4]:
found = False
key = "KaggleDSFraudDS"
description_text = "Kaggle DataSet for Udacity Course Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        df = dataset.to_pandas_dataframe()
        df.describe()

,account_age_days,transaction_amt,transaction_adj_amt,historic_velocity,billing_postal,card_bin,days_since_last_logon,inital_amount
count,149892.000000,149870.000000,149886.000000,149885.000000,149876.000000,149872.000000,149864.000000,149872.000000
mean,4645.751995,2521.458944,54.202434,4702.200033,50190.088160,41837.521512,49.866392,7998.025275
std,1153.816155,604.524161,10.070401,1188.476704,28378.078852,10080.587192,29.204766,4047.071560
min,1.000000,5.000000,1.000000,50.000000,503.000000,6040.000000,0.000000,1000.000000
25%,3824.000000,2103.000000,48.000000,3872.000000,25314.000000,35386.000000,24.000000,4487.000000
50%,4667.000000,2544.000000,55.000000,4730.000000,50117.000000,42086.000000,50.000000,8002.000000
75%,5476.000000,2951.000000,61.000000,5548.000000,74338.000000,47374.000000,75.000000,11494.000000
max,9119.000000,4999.000000,99.000000,9999.000000,99950.000000,67639.000000,100.000000,15000.000000


In [5]:
# choose a name for experiment
experiment_name = 'CapstoneExpAutoML'
project_folder = './capstone-project'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
CapstoneExpAutoML,wsaug,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

The automl model is trained for classification scenario. The experiment will run maximum for 20 minutes. It uses the given dataset and is trained to predict the class for column EVENT_LABEL. It uses AUC_Weighted metric for the fraud detection scenario.

In [7]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted' 
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="EVENT_LABEL",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
remote_run.wait_for_completion()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExpAutoML,AutoML_b797e176-5f53-48cc-8c3d-3932a649d64a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_b797e176-5f53-48cc-8c3d-3932a649d64a',
 'target': 'cluster4',
 'status': 'Completed',
 'startTimeUtc': '2022-08-09T10:42:57.000772Z',
 'endTimeUtc': '2022-08-09T11:18:20.328867Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 20 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cluster4',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExpAutoML","subscription_id":"d18bf486-2340-49bd-9908-fbd241f929a1","resource_group":"RGML","workspace_name":"wsaug","region":"centralindia","compute_target":"cluster4","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_run_rest

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run_auto=remote_run.get_best_child()
best_run_model_name=best_run_auto.properties["model_name"]
best_run_auto.properties

{'runTemplate': 'automl_child',
 'pipeline_id': 'c7af0367625be6ac5c2fecbfc72ed444cb7a2111',
 'pipeline_spec': '{"objects":[{"spec_class":"preproc","class_name":"MaxAbsScaler","module":"sklearn.preprocessing","param_args":[],"param_kwargs":{},"prepared_kwargs":{}},{"spec_class":"sklearn","class_name":"XGBoostClassifier","module":"automl.client.core.common.model_wrappers","param_args":[],"param_kwargs":{"tree_method":"auto"},"prepared_kwargs":{}}],"pipeline_id":"c7af0367625be6ac5c2fecbfc72ed444cb7a2111","module":"sklearn.pipeline","class_name":"Pipeline"}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '1',
 '_aml_system_scenario_identification': 'Remote.Child',
 'run_preprocessor': 'MaxAbsScaler',
 'run_algorithm': 'XGBoostClassifier',
 'model_output_path': 'outputs/model.pkl',
 'conda_env_data_location': 'aml://artifact/ExperimentRun/dcid.AutoML_b797e176-5f53-48cc-8c3d-3932a649d64a_1/outputs/conda_env_v_1_0_0.yml',
 'model_data_location': 'aml://artifact/Experiment

In [29]:
#TODO: Save the best model
mlmodel = best_run_auto.register_model(model_name=best_run_model_name,description='automl model',tags=None)

In [30]:
print(mlmodel)

Model(workspace=Workspace.create(name='wsaug', subscription_id='d18bf486-2340-49bd-9908-fbd241f929a1', resource_group='RGML'), name=AutoMLb797e17651, id=AutoMLb797e17651:2, version=2, tags={}, properties={})


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

The best run is deployed to ACI. The required environment settings are provided in the yml file.

In [5]:
from azureml.automl.core.shared import constants
from azureml.core import Environment

#best_run_auto.download_file(constants.CONDA_ENV_FILE_PATH, 'automlenv.yml')
automlenv = Environment.from_conda_specification(name="automlenv", file_path="automlenv.yml")

In [6]:
from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

classifier_inference_config = InferenceConfig(source_directory = '.',
                                              entry_script='score.py',
                                              environment=automlenv)
                                              

classifier_deploy_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1, auth_enabled=True)

service = Model.deploy(workspace=ws,
                       name = 'automlclassifierservice',
                       models = [mlmodel],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config                       
                       )
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-08-12 06:03:39+00:00 Creating Container Registry if not exists.
2022-08-12 06:03:42+00:00 Use the existing image.
2022-08-12 06:03:42+00:00 Generating deployment configuration.
2022-08-12 06:03:44+00:00 Submitting deployment to compute..
2022-08-12 06:03:54+00:00 Checking the status of deployment automlclassifierservice..
2022-08-12 06:06:52+00:00 Checking the status of inference endpoint automlclassifierservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [7]:
print(service.scoring_uri)

http://bd7a8fbe-c178-4141-a7be-bcbba4fb052d.centralindia.azurecontainer.io/score


In [10]:
from azureml.core.workspace import Workspace
import joblib
from azureml.core import Datastore
# import pandas as pd
# from azureml.core.run import Run
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.webservice import Webservice
import numpy as np
import requests
import json
 
data={"data":
       [{
            "account_age_days":4946,
"transaction_amt":2905,
"transaction_adj_amt":39,
"historic_velocity":4241,
"ip_address":"58.66.111.116",
"user_agent":"Mozilla/5.0 (Windows; U; Windows NT 10.0) AppleWebKit/535.17.7 (KHTML, like Gecko) Version/5.0.5 Safari/535.17.7",
"email_domain":"price-jackson.info",
"phone_number":"820.940.5485",
"billing_city":"East Amandaborough",
"billing_postal":89082,
"billing_state":"Indiana",
"card_bin":49836,
"currency":"usd",
"cvv":"W",
"signature_image":"W",
"transaction_type":"W",
"transaction_env":"X",
"EVENT_TIMESTAMP":"12",
"applicant_name":"Amy Alvarado",
"billing_address":"7874 Morris Junctions Apt. 683",
"merchant_id":"707-23-1553",
"locale":"cs_CZ",
"tranaction_initiate":"F",
"days_since_last_logon":26,
"inital_amount":9468
          }
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
key='Gd7kbtHDH0bQqqsncAO25fqY7clBUXOg'
headers['Authorization'] = f'Bearer {key}'
# Make the request and display the response
resp = requests.post(service.scoring_uri, input_data, headers=headers)
print(resp.text)


"{\"result\": [\"legit\"]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [12]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_01cff5d1570c760e840399cd816552f4/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_01cff5d1570c760e840399cd816552f4/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_01cff5d1570c760e840399cd816552f4/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_01cff5d1570c760e840399cd816552f4/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2022-08-12T06:06:33,256783200+00:00 - rsyslog/run 
2022-08-12T06:06:33,265218000+00:00 - iot-server/run 
bash: /azureml-envs/azureml_01cff5d1570c760e840399cd816552f4/lib/libtinfo.so.6: no version information available (required by bash)
2022-08-12T06:06:33,288083100+00:00 - gunicorn/run 
2022-08-12T06:06:33,290201500+00:00 - nginx/run 
2022-08-12T06:06:33,295901000+00:00 | gunicorn/run | 
2022-08-12T06:06:33,317681800+00:00 | gu

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.


In [13]:
service.delete()